# Introduction

**DISCLAIMER**: Some parts of the following code was inspired by looking at the work that was done last year about https://www.goodreads.com, for example by https://github.com/GiorgiaSalvatori/ADM-HW3/blob/main/main.ipynb. Also the following post was useful https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a.

Our goal is to build a search engine over the "Top Anime Series" from the list of MyAnimeList. There is no provided dataset, so we create our own.

In [1]:
# libraries to install
# !pip install selenium

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm # useful for progress bars
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import re
import os 

## 1. Data Collection

We start from the list of animes to include in the corpus of documents the search engine will work on. In particular, we focus on the top animes ever list: https://myanimelist.net/topanime.php.  The list is long and splitted in many pages. The first thing we will do is to retrieve the urls (and the names) of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

### 1.1 Get the list of animes

Here we will extract the *urls* and the *names* of the animes in the list. At first we can have an idea of the necessary steps to extract the informations we want by working on a single anime in the list and then proceed by iteration. 

After inspecting the HTML code of the site, we saw that the all the informations we need from a single anime are stored in  `tr` blocks inside a single `table` that contains the list of all the top animes in the site. To get the  name of an anime in the list we should work on `a` tags, whereas to get the url we need to work on `td` tags (leveraging the property `href`). 

Knowing these HTML details we can use the `selenium` library to do the web-scrapping.

In [3]:
from selenium.webdriver.chrome.service import Service

In [4]:
s = Service('/Users/dany/Desktop/adm-hw3/chromedriver')

In [5]:
# selenium with Chrome
driver = webdriver.Chrome(service=s)

In [6]:
# create a dataframe with links of each anime
df = pd.DataFrame(columns = ['Href'])

In [7]:
# go page by page and and store links in a list
anime_list = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime_list.append((link.contents[0], link.get('href')) )

100%|██████████| 400/400 [06:16<00:00,  1.06it/s]


In [ ]:
# total number of animes
print(len(anime_list))

In [10]:
# check for duplicates: ok no duplicates
df['Href'].nunique()

19124

In [9]:
# assign list to dataframe
df['Href'] = anime_list

The following code shows the first informations we have acquired.

In [12]:
df['Href'].head()

0    (Fullmetal Alchemist: Brotherhood, https://mya...
1    (Gintama°, https://myanimelist.net/anime/28977...
2    (Shingeki no Kyojin Season 3 Part 2, https://m...
3    (Steins;Gate, https://myanimelist.net/anime/92...
4    (Fruits Basket: The Final, https://myanimelist...
Name: Href, dtype: object

We save the dataframe into a *csv* file without header and comma separator. This is equivalent to a *txt* file, with not only the urls, but also the names of the animes that may be of help in some data processing stages. 

In [11]:
df.to_csv('urls.csv', sep = ' ', header=False)

We could also create a dictionary as this is useful in some circumnstances.

In [13]:
#keys
name = []   
#values
url = []    

for item in anime_list:
    name.append(item[0])
    url.append(item[1])
    
D = dict(zip(name, url))

# display first 5 urls
print(list(D.values())[0:5])

['https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood', 'https://myanimelist.net/anime/28977/Gintama°', 'https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2', 'https://myanimelist.net/anime/9253/Steins_Gate', 'https://myanimelist.net/anime/42938/Fruits_Basket__The_Final']


## 1.2 Crawl animes

We procede to:
- download the html corresponding to each of the collected urls;
- save its html in a file;
- organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

To do so we extensively use the `os` library to create directories, changing paths, etc...

In [ ]:
# remark: the execution can take quite some time
# remark: there is an issue with high frequency site-connections 

# returns current working directory
base = os.getcwd()  
t = 0
# we use the previously created dictionary D to get the urls we need
scrapped_urls = list(D.values())[0:-1]
for i in range(len(scrapped_urls)):
    if(i%50==0):
        # create a new folder
        # remark: the newley created pages will start from 0
        page_identifier = i-(49*t)
        # subdirectory 
        directory = f"page_{page_identifier}.html"
        # parent directories
        parent_dir = base
        # path
        path = os.path.join(parent_dir, directory)
        # make directory
        os.makedirs(path)
        # checkpoint
        print("Directory '%s' created" %directory)
        # change directory 
        os.chdir(path)
        t += 1

    # to avoid connection issues    
    time.sleep(5)   

    # get urls
    URL = scrapped_urls[i]
    page = requests.get(URL)
    
    # parsing
    soup_data = BeautifulSoup(page.content, "html.parser")
    
    # saving
    with open(f"article_{i}.html", "w") as file:
        file.write(str(soup_data))
        
    # checkpoint
    print(f"Article {i} successfully written!")

Directory 'page_0.html' created
Article 0 successfully written!
Article 1 successfully written!
Article 2 successfully written!
Article 3 successfully written!
Article 4 successfully written!
Article 5 successfully written!
Article 6 successfully written!
Article 7 successfully written!
Article 8 successfully written!
Article 9 successfully written!
Article 10 successfully written!
Article 11 successfully written!
Article 12 successfully written!
Article 13 successfully written!
Article 14 successfully written!
Article 15 successfully written!
Article 16 successfully written!
Article 17 successfully written!
Article 18 successfully written!
Article 19 successfully written!
Article 20 successfully written!
Article 21 successfully written!
Article 22 successfully written!
Article 23 successfully written!
Article 24 successfully written!
Article 25 successfully written!
Article 26 successfully written!
Article 27 successfully written!
Article 28 successfully written!
Article 29 successful

Article 240 successfully written!
Article 241 successfully written!
Article 242 successfully written!
Article 243 successfully written!
Article 244 successfully written!
Article 245 successfully written!
Article 246 successfully written!
Article 247 successfully written!
Article 248 successfully written!
Article 249 successfully written!
Directory 'page_5.html' created
Article 250 successfully written!
Article 251 successfully written!
Article 252 successfully written!
Article 253 successfully written!
Article 254 successfully written!
Article 255 successfully written!
Article 256 successfully written!
Article 257 successfully written!
Article 258 successfully written!
Article 259 successfully written!
Article 260 successfully written!
Article 261 successfully written!
Article 262 successfully written!
Article 263 successfully written!
Article 264 successfully written!
Article 265 successfully written!
Article 266 successfully written!
Article 267 successfully written!
Article 268 succ

Article 477 successfully written!
Article 478 successfully written!
Article 479 successfully written!
Article 480 successfully written!
Article 481 successfully written!
Article 482 successfully written!
Article 483 successfully written!
Article 484 successfully written!
Article 485 successfully written!
Article 486 successfully written!
Article 487 successfully written!
Article 488 successfully written!
Article 489 successfully written!
Article 490 successfully written!
Article 491 successfully written!
Article 492 successfully written!
Article 493 successfully written!
Article 494 successfully written!
Article 495 successfully written!
Article 496 successfully written!
Article 497 successfully written!
Article 498 successfully written!
Article 499 successfully written!
Directory 'page_10.html' created
Article 500 successfully written!
Article 501 successfully written!
Article 502 successfully written!
Article 503 successfully written!
Article 504 successfully written!
Article 505 suc

Article 714 successfully written!
Article 715 successfully written!
Article 716 successfully written!
Article 717 successfully written!
Article 718 successfully written!
Article 719 successfully written!
Article 720 successfully written!
Article 721 successfully written!
Article 722 successfully written!
Article 723 successfully written!
Article 724 successfully written!
Article 725 successfully written!
Article 726 successfully written!
Article 727 successfully written!
Article 728 successfully written!
Article 729 successfully written!
Article 730 successfully written!
Article 731 successfully written!
Article 732 successfully written!
Article 733 successfully written!
Article 734 successfully written!
Article 735 successfully written!
Article 736 successfully written!
Article 737 successfully written!
Article 738 successfully written!
Article 739 successfully written!
Article 740 successfully written!
Article 741 successfully written!
Article 742 successfully written!
Article 743 su

Article 951 successfully written!
Article 952 successfully written!
Article 953 successfully written!
Article 954 successfully written!
Article 955 successfully written!
Article 956 successfully written!
Article 957 successfully written!
Article 958 successfully written!
Article 959 successfully written!
Article 960 successfully written!
Article 961 successfully written!
Article 962 successfully written!
Article 963 successfully written!
Article 964 successfully written!
Article 965 successfully written!
Article 966 successfully written!
Article 967 successfully written!
Article 968 successfully written!
Article 969 successfully written!
Article 970 successfully written!
Article 971 successfully written!
Article 972 successfully written!
Article 973 successfully written!
Article 974 successfully written!
Article 975 successfully written!
Article 976 successfully written!
Article 977 successfully written!
Article 978 successfully written!
Article 979 successfully written!
Article 980 su

...

Article 12176 successfully written!
Article 12177 successfully written!
Article 12178 successfully written!
Article 12179 successfully written!
Article 12180 successfully written!
Article 12181 successfully written!
Article 12182 successfully written!
Article 12183 successfully written!
Article 12184 successfully written!
Article 12185 successfully written!
Article 12186 successfully written!
Article 12187 successfully written!
Article 12188 successfully written!
Article 12189 successfully written!
Article 12190 successfully written!
Article 12191 successfully written!
Article 12192 successfully written!
Article 12193 successfully written!
Article 12194 successfully written!
Article 12195 successfully written!
Article 12196 successfully written!
Article 12197 successfully written!
Article 12198 successfully written!
Article 12199 successfully written!
Directory 'page_244.html' created
Article 12200 successfully written!
Article 12201 successfully written!
Article 12202 successfully wri

Article 12400 successfully written!
Article 12401 successfully written!
Article 12402 successfully written!
Article 12403 successfully written!
Article 12404 successfully written!
Article 12405 successfully written!
Article 12406 successfully written!
Article 12407 successfully written!
Article 12408 successfully written!
Article 12409 successfully written!
Article 12410 successfully written!
Article 12411 successfully written!
Article 12412 successfully written!
Article 12413 successfully written!
Article 12414 successfully written!
Article 12415 successfully written!
Article 12416 successfully written!
Article 12417 successfully written!
Article 12418 successfully written!
Article 12419 successfully written!
Article 12420 successfully written!
Article 12421 successfully written!
Article 12422 successfully written!
Article 12423 successfully written!
Article 12424 successfully written!
Article 12425 successfully written!
Article 12426 successfully written!
Article 12427 successfully w

Article 12624 successfully written!
Article 12625 successfully written!
Article 12626 successfully written!
Article 12627 successfully written!
Article 12628 successfully written!
Article 12629 successfully written!
Article 12630 successfully written!
Article 12631 successfully written!
Article 12632 successfully written!
Article 12633 successfully written!
Article 12634 successfully written!
Article 12635 successfully written!
Article 12636 successfully written!
Article 12637 successfully written!
Article 12638 successfully written!
Article 12639 successfully written!
Article 12640 successfully written!
Article 12641 successfully written!
Article 12642 successfully written!
Article 12643 successfully written!
Article 12644 successfully written!
Article 12645 successfully written!
Article 12646 successfully written!
Article 12647 successfully written!
Article 12648 successfully written!
Article 12649 successfully written!
Directory 'page_253.html' created
Article 12650 successfully wri

Article 12848 successfully written!
Article 12849 successfully written!
Directory 'page_257.html' created
Article 12850 successfully written!
Article 12851 successfully written!
Article 12852 successfully written!
Article 12853 successfully written!
Article 12854 successfully written!
Article 12855 successfully written!
Article 12856 successfully written!
Article 12857 successfully written!
Article 12858 successfully written!
Article 12859 successfully written!
Article 12860 successfully written!
Article 12861 successfully written!
Article 12862 successfully written!
Article 12863 successfully written!
Article 12864 successfully written!
Article 12865 successfully written!
Article 12866 successfully written!
Article 12867 successfully written!
Article 12868 successfully written!
Article 12869 successfully written!
Article 12870 successfully written!
Article 12871 successfully written!
Article 12872 successfully written!
Article 12873 successfully written!
Article 12874 successfully wri

Article 13071 successfully written!
Article 13072 successfully written!
Article 13073 successfully written!
Article 13074 successfully written!
Article 13075 successfully written!
Article 13076 successfully written!
Article 13077 successfully written!
Article 13078 successfully written!
Article 13079 successfully written!
Article 13080 successfully written!
Article 13081 successfully written!
Article 13082 successfully written!
Article 13083 successfully written!
Article 13084 successfully written!
Article 13085 successfully written!
Article 13086 successfully written!
Article 13087 successfully written!
Article 13088 successfully written!
Article 13089 successfully written!
Article 13090 successfully written!
Article 13091 successfully written!
Article 13092 successfully written!
Article 13093 successfully written!
Article 13094 successfully written!
Article 13095 successfully written!
Article 13096 successfully written!
Article 13097 successfully written!
Article 13098 successfully w

### 1.3 Parse downloaded pages

At this point we have all the html documents about the animes of interest and we can start to extract the animes informations:
- Anime Name (to save as `animeTitle`): String
-Anime Type (to save as `animeType`): String
-Number of episode (to save as `animeNumEpisode`): Integer
-Release and End Dates of anime (to save as `releaseDate` and `endDate`): Convert both release and end date into datetime format.
-Number of members (to save as `animeNumMembers`): Integer
-Score (to save as `animeScore`): Float
-Users (to save as `animeUsers`): Integer
-Rank (to save as `animeRank`): Integer
-Popularity (to save as `animePopularity`): Integer
-Synopsis (to save as `animeDescription`): String
-Related Anime (to save as `animeRelated`): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.
-Characters (to save as `animeCharacters`): List of strings.
-Voices (to save as `animeVoices`): List of strings
-Staff (to save as `animeStaff`): Include the staff name and their responsibility/task in a list of lists.

In [ ]:
animeTitle = []
animeType = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []


# I still don't know how to integrate directory paths
# The following code should work only with files
# For the sake of simplicity the following code works for the first 5 files
# article_0, ...., article_5
# Attention: There are some bugs 
for i in range(5):
    # anime titles as test case above
    # a similar code can be used for the other informations
    path = "article_" + str(i) + ".html"
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left of the html page
    # e.g. for article_0 under Alternative Titles we have English: Fullmetal Alchemist: Brotherhood 
    # that corresponds to 
    # <div class="spaceit_pad"> Fullmetal Alchemist: Brotherhood <span class="dark_text">English:</span></div>
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # TYPES
            if span.contents[0] == 'Type:':
                animeType.append(div.find_all('a')[0].contents[0])
            # NUMBER OF EPISODES
            if span.contents[0] == 'Episodes:':
                animeNumEpisode.append(int(div.contents[2]))
            # DATES
            if span.contents[0] == 'Aired:':
                if len(div.contents[2]) > 21:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:16], errors ='coerce', format='%m%d%Y'))
                    endDate.append(pd.to_datetime(div.contents[2][19:-4], errors ='coerce', format='%m%d%Y'))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][4:-4], errors ='coerce', format='%m%d%Y'))
                    endDate.append('-')
                    
    # center of the html page
    # similar to what was done before
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        
        # MEMBERS
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
    
        # USERS
        users = div.find_all("div", {"class": "fl-l score"})
        # here we we eliminate the word 'user '   
        # that is why there is the [:-6] part
        # we also replace the comma divisor
        animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        
        
        # RANK
        rank = div.find_all("span", {"class": "numbers ranked"})
        animeRank.append(int(rank[0].contents[1].contents[0][1:]))
       
    
        # POPULARITY
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
    
    
    
    # DESCRIPTION
    # center of the html page
    animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
     
    
    # RELATED 
    # center of the html page
    x = []
    y = []
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            t = td[i+1].find_all("a")
            y.append(t[0].contents[0])
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
    
    
    # CHARACTERS
    # center of the html page (bottom
    characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
    x = []
    for i in chars:
        x.append(i.contents[0].contents[0])
    animeCharacters.append(x)
    
    
    # VOICES
    # center of the html page (bottom)
    voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
    y = []
    for i in voices:
        y.append(i.contents[1].contents[0])
    animeVoices.append(y)

    
    # STAFF
    # center of the html page (bottom)
    staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    staff = staff[1].find_all("td")
    x = []
    y = []
    for i in range(1, len(staff), 2):
        x.append(staff[i].contents[1].contents[0])
        y.append(staff[i].find_all("small")[0].contents[0])
    animeStaff.append([list(i) for i in list(zip(x,y))])    

    
# check 
print(releaseDate)
print(endDate)
print(animeUsers)
print(animePopularity)
print(animeDescription)
print(animeRelated)
print(animeCharacters)
print(animeVoices)
print(animeStaff)

In [ ]:
dataset = pd.DataFrame(
    [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, 
     animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, 
     animeCharacters, animeVoices, animeStaff], 
    index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 
           'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']).T
dataset.head()

In [ ]:
import csv
import os.path
save_path = 'simplePath'

In [ ]:
# for each row create a tsv file
for i in range(len(dataset)):
    with open('anime_tsv/anime_'+str(i)+'.tsv', 'wt') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in df.columns]) #the header row
        tsv_writer.writerow(x for x in df.iloc[i]) #the value under each columns